In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import transforms
import torchvision.models as models
import pandas as pd
from PIL import Image
import math
import pickle

In [12]:
LR = 0.01
BATCH_SIZE = 16
EPOCH = 100000
DATA_FOLDER = 'set02'
#model_path = './Resnet/model/10-Res18.pkl'

In [13]:
Train_Loss_d = []
Train_Loss_phi = []
Train_Loss = []
Test_Loss_d = []
Test_Loss_phi = []
Test_Loss = []

In [14]:
def getData(mode):
    if mode == 'train':
        
        img = pd.read_csv(DATA_FOLDER + '_train_img.csv')
        label = pd.read_csv(DATA_FOLDER + '_train_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv(DATA_FOLDER + '_test_img.csv')
        label = pd.read_csv(DATA_FOLDER + '_test_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)


class Loader(Data.Dataset):
    def __init__(self, root, mode):
        """
        Args:
            root (string): Root path of the dataset.
            mode : Indicate procedure status(training or testing)

            self.img_name (string list): String list that store all image names.
            self.label (int or float list): Numerical list that store all ground truth label values.
        """
        self.root = root
        self.img_name, self.label = getData(mode)
        self.mode = mode
        print("> Found %d images..." % (len(self.img_name)))
        

    def __len__(self):
        #------------return the size of dataset
        return len(self.img_name)

    def __getitem__(self, index):
        #-------------Get the image path from 'self.img_name' and load it.
                  
        path = self.root + str(self.img_name[index]) + '.png'
        img = Image.open(path)
        #img = img.resize(,Image.ANTIALIAS)
        box=(0,120,640,480)
        img_as_img=img.crop(box)
        
        #-------------Get the ground truth label from self.label"""
        label = torch.from_numpy(self.label)[index]
        
        #-------------Transform the .jpeg rgb images
        if self.mode == 'train':
            transform1 = transforms.Compose([
#                 transforms.RandomHorizontalFlip(p=0.5),
#                 transforms.RandomVerticalFlip(p=0.5),
#                 transforms.RandomRotation(degrees=(-45,45), resample=False, expand=False, center=None),
#                 transforms.ColorJitter(contrast=(0,1)),
                transforms.Resize(( 320, 180)),
                transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                #transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5)),#range [0, 255] ->[-1.0,1.0]
                
                ]
            )
        else:
            transform1 = transforms.Compose([
                transforms.Resize(( 320, 180)),
                transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                #transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5)),
             
                ]
            )
        img_tran = transform1(img_as_img)
                
        #-------------Return processed image and label
        return img_tran, label

In [15]:
# Dataloader

train_data=Loader('./'+ DATA_FOLDER +'/','train')
test_data=Loader('./'+ DATA_FOLDER +'/','test')
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE)
test_loader = Data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
print (train_loader)
print (test_loader)

> Found 23399 images...
> Found 2599 images...


In [16]:
def train(epoch):
    model_ResNet18.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to(device=device, dtype=torch.float), target.to(device)

        optimizer.zero_grad()
        output = model_ResNet18(data)
        target = target.float()
        loss_d = Loss(output[0], target[0])
        loss_phi = Loss(output[1], target[1])
        loss = Loss(output, target)
        loss.backward()
        optimizer.step()
        if( (batch_idx%100) == 0 ):
            print("===train output\n" ,output.cpu().detach().numpy().squeeze()[0:8])
            print("ground truth\n" ,target.cpu().detach().numpy().squeeze()[0:8])
    Train_Loss.append(round((loss.data).cpu().numpy().tolist(),6))
    Train_Loss_d.append(round((loss_d.data).cpu().numpy().tolist(),6))
    Train_Loss_phi.append(round((loss_phi.data).cpu().numpy().tolist(),6))
   
    print('Train Epoch: {} \t Loss_d: {} Loss_phi:{}'.format(
            epoch, loss_d.data , loss_phi.data))

In [17]:
def test(epoch):
    model_ResNet18.eval()
    test_loss_d = 0.0
    test_loss_phi = 0.0
    
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if torch.cuda.is_available():
            data, target = data.to(device=device, dtype=torch.float), target.to(device)
        with torch.no_grad():
            output = model_ResNet18(data)
        target = target.float()
        test_loss_d = Loss(output[0], target[0])
        test_loss_phi = Loss(output[1], target[1])
    
    
    print(" ground truth\n" ,target.cpu().detach().numpy().squeeze()[0:8])
    print(output.cpu().detach().numpy().squeeze()[0:8])
    
    Test_Loss_d.append(round((test_loss_d.data).cpu().numpy().tolist(),6))
    Test_Loss_phi.append(round((test_loss_phi.data).cpu().numpy().tolist(),6))
        
    print('========Test set: Average loss d:{}\t loss phi:{}========\n'
          .format(test_loss_d,test_loss_phi))

In [18]:
model_ResNet18 = models.resnet18(pretrained=True).cuda(0)
model_ResNet18.fc=nn.Linear(512 * 1, 2 ,bias=True)

if torch.cuda.is_available():
    device = torch.device('cuda')
    model_ResNet18.to(device) 
#------------define optimizer/loss function

# if(model_path):
#     model_ResNet18.load_state_dict(torch.load(model_path))
#     model_ResNet18.eval()
#     print("load model:" ,model_path )
Loss = nn.MSELoss(reduction='mean')    
optimizer = torch.optim.SGD(model_ResNet18.parameters(), lr=LR, momentum=0.9, 
                            dampening=0, weight_decay=0.0005, nesterov=False)
print(model_ResNet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [ ]:
def savelossfile():
    print("save loss ")
    # pickle a variable to a file
    file = open('./Resnet/Train_loss_d.pickle', 'wb')
    pickle.dump(Train_Loss_d, file)
    file.close()
    # pickle a variable to a file
    file = open('./Resnet/Train_loss_phi.pickle', 'wb')
    pickle.dump(Train_Loss_phi, file)
    file.close()
    # pickle a variable to a file
    file = open('./Resnet/Train_loss.pickle', 'wb')
    pickle.dump(Train_Loss, file)
    file.close()
    # pickle a variable to a file
    file = open('./Resnet/Test_loss_d.pickle', 'wb')
    pickle.dump(Test_Loss_d, file)
    file.close()
    # pickle a variable to a file
    file = open('./Resnet/Test_loss_phi.pickle', 'wb')
    pickle.dump(Test_Loss_phi, file)
    file.close()

In [ ]:
savelossfile()
#---------------Train-----------------
for epoch in range(1,EPOCH+1):
    train(epoch)
    test(epoch)
    if (epoch%10==0):
        PATH ='./Resnet/model/'+str(epoch)+'-Res18.pkl'
        torch.save(model_ResNet18.state_dict(), PATH)
        savelossfile()
    

save loss 
===train output
 [[-0.18494298  0.01857511]
 [-0.469923   -0.47743234]
 [ 0.6455534  -0.24572462]
 [-0.07974804 -0.4931551 ]
 [-0.41955724 -0.3515581 ]
 [ 0.58498615 -0.8637112 ]
 [-0.1266373  -0.8587911 ]
 [ 0.08428226 -0.6453409 ]]
ground truth
 [[-0.32360402 -0.2308306 ]
 [-0.00863334  0.11166698]
 [ 0.08791935  0.7324858 ]
 [-0.28524333  0.8343907 ]
 [-0.12443631 -0.8520457 ]
 [-0.29704875  0.00839745]
 [-0.04447002 -0.56124216]
 [-0.2998994  -0.01855909]]
===train output
 [[-0.02406181  0.17576301]
 [-0.02867852  0.17347187]
 [-0.03032049  0.17481379]
 [-0.02885413  0.17512767]
 [-0.02933558  0.1748508 ]
 [ 0.2827529   0.7985148 ]
 [ 0.08733493  0.6533434 ]
 [-0.02949175  0.17438741]]
ground truth
 [[-0.03743719 -0.3349506 ]
 [ 0.03523051  0.5671594 ]
 [-0.2212894  -0.45890236]
 [-0.33218116 -0.4558895 ]
 [-0.07312914 -0.89939934]
 [ 0.03971943  0.48747718]
 [-0.08407466  0.6336717 ]
 [-0.14970024 -0.03847773]]
===train output
 [[-0.07138915  0.43564895]
 [-0.11080177  

===train output
 [[-0.15763406  0.2968956 ]
 [-0.10517957  0.23232645]
 [-0.11183354  0.7036188 ]
 [-0.18294045 -0.14495668]
 [-0.14795765  0.01165621]
 [-0.15481472  0.44784614]
 [-0.16015092 -0.06429768]
 [-0.10065158  0.45010227]]
ground truth
 [[ 0.03352918 -0.05467312]
 [ 0.05440861  0.89172894]
 [-0.00144823  0.39524388]
 [-0.33759907 -0.58826894]
 [-0.2692169  -1.0087788 ]
 [-0.10668713  0.17468013]
 [ 0.07181884 -0.56341946]
 [-0.16937542 -0.39954877]]
===train output
 [[-0.1225439  -0.404401  ]
 [-0.03906278 -0.09259419]
 [-0.08263034 -0.17915523]
 [-0.07373844 -0.12595122]
 [-0.08692048 -0.06715657]
 [-0.11796263 -0.4116349 ]
 [-0.09234905 -0.15038446]
 [-0.13248815 -0.5152046 ]]
ground truth
 [[-0.1728886  -0.8136686 ]
 [-0.02366172 -0.8412634 ]
 [-0.15990138 -0.32002586]
 [-0.12669325  0.82870996]
 [ 0.03481191  0.48243466]
 [-0.2319947  -0.0434593 ]
 [ 0.11449871 -0.16778357]
 [-0.17912444 -0.8611567 ]]
===train output
 [[-0.05223591  0.5285837 ]
 [-0.08688186  0.43903625]

===train output
 [[-0.01593906  0.6498404 ]
 [-0.03552298  0.55541456]
 [-0.05712454  0.6518377 ]
 [-0.2032357   0.62516665]
 [-0.23716949 -0.03689507]
 [-0.1690961   0.78752565]
 [-0.06993628  0.01789148]
 [-0.02023214  0.70295435]]
ground truth
 [[ 0.006334    1.0427899 ]
 [ 0.0348395   0.8970293 ]
 [-0.1079201   0.9833796 ]
 [-0.31244075  0.8308631 ]
 [-0.3401581  -0.8202379 ]
 [-0.26773873 -0.6957776 ]
 [-0.13708253  0.3733869 ]
 [-0.01529757  0.5574338 ]]
===train output
 [[ 0.01291887  0.61371857]
 [-0.2702912  -0.5830252 ]
 [-0.10965402 -0.9759949 ]
 [-0.06627318 -1.0776522 ]
 [-0.1686057   0.7133735 ]
 [-0.09206833 -0.20326659]
 [-0.10332167 -0.88558394]
 [-0.10284404  0.06898303]]
ground truth
 [[ 0.07227621  0.6340983 ]
 [-0.2661144  -0.5848569 ]
 [-0.11521424 -0.87524176]
 [ 0.01121056 -1.0179248 ]
 [-0.135255    0.85689664]
 [-0.1949584  -0.14316845]
 [ 0.00812333 -1.0467417 ]
 [-0.17263888  0.06652143]]
===train output
 [[ 0.15337361  0.5653064 ]
 [-0.34538686 -0.6872572 ]

===train output
 [[ 0.10111697  0.69074565]
 [-0.2190364  -0.6436834 ]
 [-0.07336394 -0.6675258 ]
 [-0.09866307  0.30243653]
 [-0.06990345  0.2629583 ]
 [-0.10169433  0.42648023]
 [ 0.10925801 -0.48917177]
 [ 0.20537011  0.04811686]]
ground truth
 [[ 0.05082829  0.5769198 ]
 [-0.31490937 -0.98985064]
 [-0.05782534 -0.55391556]
 [ 0.2145868   1.008867  ]
 [-0.11990693  1.0300211 ]
 [-0.20799726  0.6463722 ]
 [ 0.10261108 -0.40701768]
 [ 0.13157694  0.06014711]]
===train output
 [[-0.19628617 -1.0430477 ]
 [ 0.07913583  0.30490106]
 [-0.04157615 -0.8551024 ]
 [-0.215045   -0.2884544 ]
 [-0.01607893 -0.7383664 ]
 [-0.04291444  0.6092798 ]
 [-0.07675707  0.55220807]
 [-0.14271289 -0.08703347]]
ground truth
 [[-0.22660285 -0.97541046]
 [ 0.05096684  0.4504026 ]
 [-0.07592416 -0.5173468 ]
 [-0.2903598  -0.04602354]
 [-0.09261409 -0.46134728]
 [-0.09096152  0.7370271 ]
 [-0.13988066  0.9131186 ]
 [-0.22576946  0.32051805]]
===train output
 [[-0.06099188  0.8181552 ]
 [-0.19510163 -0.45476305]

===train output
 [[-0.08422496  0.7613371 ]
 [-0.19158593 -0.5256747 ]
 [-0.15290734  0.32924074]
 [-0.18413116 -0.41764563]
 [-0.16565244  0.4582165 ]
 [-0.09796769  0.770318  ]
 [-0.15206718  0.3026309 ]
 [-0.16120929 -0.55383646]]
ground truth
 [[-0.16415197  0.6353559 ]
 [-0.22918952 -0.6438243 ]
 [-0.2062452   0.21997975]
 [-0.25413677 -0.8958939 ]
 [-0.24687926  0.40949988]
 [-0.16198818  0.73130524]
 [-0.21868551  0.17525262]
 [-0.19373648 -0.59107995]]
===train output
 [[-0.13388756  0.6269995 ]
 [-0.0583963  -0.70941883]
 [-0.13081853 -0.03229493]
 [ 0.05959463  0.958119  ]
 [-0.14135209 -0.36908674]
 [-0.24251652 -0.24025178]
 [-0.13555956  0.37687778]
 [-0.10771794  0.62728107]]
ground truth
 [[-0.09190263  0.24305075]
 [-0.09441324 -0.8221079 ]
 [-0.13554747 -0.82355624]
 [ 0.05917144  0.8047604 ]
 [-0.19479544 -0.6008878 ]
 [-0.21131209 -0.7813868 ]
 [ 0.07021454  0.92771053]
 [-0.08347154  0.72997856]]
===train output
 [[ 0.01884388  0.05419102]
 [-0.46439347  0.6409935 ]